# Note book to create html pages for countries and Kreise in Germany

New strategy (with pelican)

- put notebooks into wwwroot/ipynb folder
- put html into html wwwroot/folder
- pelican files can then go into wwwroot folder

Advantages:
- cleaner than all in one folder
- github can display all files in each subdirectory (there is a limit of 500 files or so)

In [1]:
%config InlineBackend.figure_formats = ['svg']
%cp -v ../coronavirus.py .
from coronavirus import *
# force download of new data
clear_cache()

../coronavirus.py -> ./coronavirus.py


In [2]:
# Also delete date in place where notebooks are executed
!rm -rf wwwroot/ipynb/cachedir

In [3]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

Downloaded data: last data point 4/12/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
Downloaded data: last data point 4/12/20 from https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


In [4]:
def modify_template(templatefile, output_file_name, mappings, wwwroot):
    """Create concrete *.ipynb file from template
    - templatefile: the template with placeholders to be substituted
    - mappings: dictiorany with placeholders as keys, and values to be substituted
    - output_file_name: name to write modified file to
    - wwwroot: directory in which the output file should be written
    """
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(os.path.join(wwwroot, output_file_name), "tw") as f:
        f.write(template)
    print(f"Written file to {output_file_name}")


In [5]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def germany_check_region_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Bundesland'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Bundesland'].drop_duplicates()))}"

def germany_check_subregion_name_is_known(name):
    d = fetch_data_germany()
    assert name in list(d['Landkreis'].drop_duplicates()), \
        f"{name} is unknown. Known regions are {sorted(list(d['Landkreis'].drop_duplicates()))}"

germany_check_region_name_is_known("Hamburg") 
germany_check_subregion_name_is_known("SK Hamburg") 

    
def sanitise(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    
    
def get_binder_url(notebook):
    """Given a notebook name, compute the path"""
    base = "https://mybinder.org/v2/gh/fangohr/coronavirus/master?filepath=ipynb/"
    return base + notebook.replace(" ", "%20")


def create_ipynb_for_country(country, templatename, wwwroot):
    """Creates ipynb file for country, based on templatename. 
    File is based in ipynb subfolder of wwwroot.
    Returns name of file."""
    
    # create ipynb folder if required
    ipynb_dir = os.path.join(wwwroot, "ipynb")
    if not os.path.exists(ipynb_dir):
        os.mkdir(ipynb_dir)
        
    
    check_country_name_is_known(country)
    
    output_file_name =  f"{country}.ipynb"
    output_file_path = os.path.join(wwwroot, "ipynb", output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "",
        "%country%" : country,
        "%binderurl%" : get_binder_url(output_file_name),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    }

    modify_template(templatename, os.path.join("ipynb", output_file_name), mappings, wwwroot)
    assert os.path.exists(output_file_path), f"{output_file_path} does not exist"
    return output_file_name

def create_ipynb_for_germany(region, subregion, templatename, wwwroot):
    """Creates ipynb file for region and subregion in Germany, based on templatename. 
    File is based in ipynb subfolder of wwwroot.
    Returns name of file."""
    germany_check_region_name_is_known(region)
    germany_check_subregion_name_is_known(subregion)
    
    output_file_name =  f"Germany-{sanitise(region)}-{sanitise(subregion)}.ipynb"
    output_file_path = os.path.join(wwwroot, "ipynb", output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : f"Germany: {subregion} ({region})",
        "%title2%" : "",
        "%region%" : region,
        "%subregion%" : subregion,
        "%binderurl%" : get_binder_url(output_file_name),
        "%create_date%" : datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    }

    modify_template(templatename, os.path.join("ipynb", output_file_name), mappings, wwwroot)
    assert os.path.exists(output_file_path), f"{output_file_path} does not exist"
    return output_file_name


Please be patient - downloading data from https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv ...
Completed downloading 91471 rows in 12.0 seconds.


In [6]:
import subprocess
def nbconvert_ipynb2html(ipynb_name, wwwroot):
    """Given the name of a a notebook (such as "germany.ipynb"), create the 
    corresponding html file ("html/germany.html") from the notebook file in 
    "ipynb" and return the name of the file (i.e. germany.html).
    """
    ipynb_dir = os.path.join(wwwroot, "ipynb")
    
    # copy file to run the notebook
    command = f"cp -fv ../coronavirus.py {ipynb_dir}"
    subprocess.check_call(command, shell=True)
    
    # copy requirements (needed for binder)
    command = f"cp -fv ../requirements.txt {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # execute notebook and create html copy from it
    command = f"""jupyter-nbconvert "{os.path.join(ipynb_dir, ipynb_name)}" """ + \
              f""" --to html --execute --output-dir {os.path.join(wwwroot, "html")}"""
    print(f"Command = {command}")
    output = subprocess.check_call(command, shell=True)

    # compute output path
    output_file_name = os.path.splitext(ipynb_name)[0] + ".html"
    assert os.path.exists(os.path.join(wwwroot, "html", output_file_name))
    
    return output_file_name
    

In [7]:
def create_md_index_list(title, links):
    """Turn dictionary of links into markdown string"""
    lines = []
    lines.append(title)
    lines.append("")   # need empty line for markdown syntax
    for name, (name_html, name_ipynb) in links.items():
        path_html = os.path.join('html', name_html)
        lines.append(f"* [{name}]({path_html})")
    return "\n".join(lines)


def test_create_md_index_list():
    title = "Title"
    links = {"France" : ("France.html", "France.ipynb")}
    assert create_md_index_list(title, links) == "Title\n\n* [France](html/France.html)"
    
test_create_md_index_list()

In [8]:
def get_country_list():
    d, c = fetch_deaths(), fetch_cases()

    countries = d.index
    countries2 = c.index
    assert (countries2 == countries).all()
    
    # Here we should identify regions in countries, and process those.
    # Instead, as a quick hack to get started, we'll just take one country
    # and the current "get_country" method will sum over all regions of one country if only 
    # the country name is given.
    
    return sorted(countries.drop_duplicates())
    

In [9]:
def create_index_page(sections, rootname, wwwroot):
    """Sections is dictionary: key is title, value is markdown text"""
    md_file = rootname + ".md"
    
    with open(os.path.join(wwwroot, md_file), "tw") as f:
        for section in sections:
            f.write(f"# {section}\n\n")
            f.write(sections[section])
    print(f"Written overview to {md_file}.")
    html_file = rootname + ".html"
    subprocess.check_call(f"pandoc -t html -o {os.path.join(wwwroot, html_file)} " +
                          f"{os.path.join(wwwroot, md_file)}", shell=True)
    return html_file

In [10]:
def get_germany_subregion_list():
    """returns list of subregions (Kreise), 
    ordered according to (i) Land, then (ii) Kreis
    """
    x = fetch_data_germany()
    land_kreis = x[['Bundesland', 'Landkreis']]
    ordered = land_kreis.sort_values(['Bundesland', 'Landkreis'])
    return list(ordered['Landkreis'].drop_duplicates())
 

@joblib_memory.cache
def germany_get_bundesland_from_kreis(kreis):
        x = fetch_data_germany()
        return x[x['Landkreis'] == kreis].iloc[0]['Bundesland']    

In [11]:
def does_wwwroot_exist(wwwroot):
    if not os.path.exists(wwwroot):
        msg = "To put the html into github repo for webhosting, run "
        msg += '"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar'
        # os.mkdir(wwwroot)
        raise ValueError(f"directory {wwwroot} missing.")

In [12]:
def create_html_for_john_hopkins_countries(countries, wwwroot):
    """countries: list of strings with countrie names
    
    returns dictionary: keys are countrynames, values are tuples with path to html file and path to notebook"""

    start_time = time.time()
    does_wwwroot_exist(wwwroot)
    
    created_files = {}
    
    for i, country in enumerate(countries):
        print(f"Processing {i+1}/{len(countries)} [{time.time()-start_time:4.0f}s]")
        ipynb_name = create_ipynb_for_country(country, "template-country.ipynb", wwwroot=wwwroot)
        html_name = nbconvert_ipynb2html(ipynb_name, wwwroot=wwwroot)
        created_files[country] = html_name, ipynb_name
    print("Create {len(countries) notebooks and html versions in" + \
          "{time.time()-start_time} seconds}")
    return created_files

In [13]:
def create_markdown_index_page(md_content, title, pelican_file_path, 
                               save_as, wwwroot):
    """Create pelican markdown file, like this:
    
    title: Germany
    category: Data
    tags: data, plots
    save-as: germany
    date: 2020-04-11 08:00
    """

    with open(os.path.join(pelican_file_path), "tw") as f:
        f.write(f"title: {title}\n")
        f.write(f"category: Data\n")
        f.write(f"tags: data, plots\n")
        f.write(f"save-as: {save_as}\n")
        date_time = datetime.datetime.now().strftime("%Y/%m/%d %H:%M")
        f.write(f"date: {date_time}\n")
        f.write("\n")
        f.write("\n")
        f.write(md_content)
        f.write("\n")


# Create country overview for the world

In [14]:
wwwroot = "wwwroot"
countries = get_country_list()
created_files = create_html_for_john_hopkins_countries(countries, wwwroot)
index_md = create_md_index_list("Countries", created_files)
create_markdown_index_page(index_md, title="World", 
                           pelican_file_path="pelican/content/world.md", save_as="world", 
                           wwwroot=wwwroot)


Processing 1/185 [   0s]
Written file to ipynb/Afghanistan.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Afghanistan.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 2/185 [   7s]
Written file to ipynb/Albania.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Albania.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 3/185 [  14s]
Written file to ipynb/Algeria.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Algeria.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 4/185 [  20s]
Written file to ipynb/Andorra.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Andorra.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 5/185 [  26s]
Written file to ipynb/Angola.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Angola.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 6/185 [  32s]
Written file to ipynb/Antigua and Barbuda.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Antigua and Barbuda.ipynb"  --to html --execute 

Processing 50/185 [ 303s]
Written file to ipynb/Dominica.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Dominica.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 51/185 [ 309s]
Written file to ipynb/Dominican Republic.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Dominican Republic.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 52/185 [ 314s]
Written file to ipynb/Ecuador.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Ecuador.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 53/185 [ 320s]
Written file to ipynb/Egypt.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Egypt.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 54/185 [ 326s]
Written file to ipynb/El Salvador.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/El Salvador.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 55/185 [ 332s]
Written file to ipynb/Equatorial Guinea.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Equatorial Guinea.ipyn

Processing 99/185 [ 582s]
Written file to ipynb/Libya.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Libya.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 100/185 [ 588s]
Written file to ipynb/Liechtenstein.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Liechtenstein.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 101/185 [ 593s]
Written file to ipynb/Lithuania.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Lithuania.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 102/185 [ 599s]
Written file to ipynb/Luxembourg.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Luxembourg.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 103/185 [ 605s]
Written file to ipynb/MS Zaandam.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/MS Zaandam.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 104/185 [ 611s]
Written file to ipynb/Madagascar.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Madagascar.ipynb"  --to html

Processing 147/185 [ 857s]
Written file to ipynb/Senegal.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Senegal.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 148/185 [ 862s]
Written file to ipynb/Serbia.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Serbia.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 149/185 [ 868s]
Written file to ipynb/Seychelles.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Seychelles.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 150/185 [ 874s]
Written file to ipynb/Sierra Leone.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Sierra Leone.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 151/185 [ 879s]
Written file to ipynb/Singapore.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Singapore.ipynb"  --to html --execute --output-dir wwwroot/html
Processing 152/185 [ 885s]
Written file to ipynb/Slovakia.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Slovakia.ipynb"  --to html --execut

# Create list of Germany data sets

In [15]:
def create_html_for_Germany(subregions, wwwroot):
    does_wwwroot_exist(wwwroot)
    start_time = time.time()
    created_files = {}

    for i, kreis in enumerate(subregions):
        bundesland = germany_get_bundesland_from_kreis(kreis)
        print(f"Processing {i+1}/{len(subregions)} [{time.time()-start_time:4.0f}s]")
        ipynb_name = create_ipynb_for_germany(region=bundesland, subregion=kreis, 
                                              templatename="template-germany.ipynb", wwwroot=wwwroot)
        html_name = nbconvert_ipynb2html(ipynb_name, wwwroot=wwwroot)
        one_line_summary = f"Germany: {bundesland} : {kreis}"
        created_files[one_line_summary] = html_name, ipynb_name
        print("Create {len(subregions) notebooks and html versions in" + \
              "{time.time()-start_time} seconds}")
    return created_files

In [35]:
wwwroot = "wwwroot"
subregions = get_germany_subregion_list()

# data cleaning: on 13 April, we had a Landkreis "LK Göttingen (alt)"
# with only one data point. This causes plots to fail, because there
# is nothing to plot, and then the legend() command failed.
# We assume that the RKI labels unusual data with '(alt)', and remove those.

alt_data_sets = [x for x in subregions if "(alt)" in x.lower()]
if len(alt_data_sets) > 0:
    print(f"Removing datasets label with '(alt)': {alt_data_sets}")
    for alt in alt_data_sets:
        c, d = germany_get_region(landkreis=alt)
        print(f"  removed: {alt} : len(cases)={len(c)}, len(deaths)={len(d)}")
    subregions = [x for x in subregions if not "(alt)" in x.lower()]

created_files = create_html_for_Germany(subregions, wwwroot)
index_md = create_md_index_list("Landkreise in Germany", 
                                created_files)
create_markdown_index_page(index_md, title="Germany", 
                           pelican_file_path="pelican/content/germany.md", save_as="germany", 
                           wwwroot=wwwroot)




Removing ['LK Göttingen (alt)']
LK Göttingen (alt) : len(cases)=1, len(deaths)=1
Processing 1/412 [   0s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Alb-Donau-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Alb-Donau-Kreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 2/412 [   8s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Biberach.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Biberach.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 3/412 [  15s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Bodenseekreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Bodenseekreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and htm

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 27/412 [ 198s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Rhein-Neckar-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Rhein-Neckar-Kreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 28/412 [ 205s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Rottweil.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Rottweil.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 29/412 [ 213s]
Written file to ipynb/Germany-Baden-Württemberg-LK-Schwarzwald-Baar-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Baden-Württemberg-LK-Schwarzwald-Baar-Kreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create 

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 53/412 [ 395s]
Written file to ipynb/Germany-Bayern-LK-Bamberg.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Bamberg.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 54/412 [ 402s]
Written file to ipynb/Germany-Bayern-LK-Bayreuth.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Bayreuth.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 55/412 [ 410s]
Written file to ipynb/Germany-Bayern-LK-Berchtesgadener-Land.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Berchtesgadener-Land.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 81/412 [ 606s]
Written file to ipynb/Germany-Bayern-LK-Landshut.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Landshut.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 82/412 [ 613s]
Written file to ipynb/Germany-Bayern-LK-Lichtenfels.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Lichtenfels.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 83/412 [ 621s]
Written file to ipynb/Germany-Bayern-LK-Lindau.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Lindau.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 84/412 [ 628s]
Writt

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 109/412 [ 815s]
Written file to ipynb/Germany-Bayern-LK-Tirschenreuth.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Tirschenreuth.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 110/412 [ 823s]
Written file to ipynb/Germany-Bayern-LK-Traunstein.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Traunstein.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 111/412 [ 832s]
Written file to ipynb/Germany-Bayern-LK-Unterallgäu.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Bayern-LK-Unterallgäu.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing

...

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 295/412 [3362s]
Written file to ipynb/Germany-Nordrhein-Westfalen-SK-Krefeld.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Nordrhein-Westfalen-SK-Krefeld.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 296/412 [3370s]
Written file to ipynb/Germany-Nordrhein-Westfalen-SK-Köln.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Nordrhein-Westfalen-SK-Köln.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 297/412 [3377s]
Written file to ipynb/Germany-Nordrhein-Westfalen-SK-Leverkusen.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Nordrhein-Westfalen-SK-Leverkusen.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html ver

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 321/412 [3557s]
Written file to ipynb/Germany-Rheinland-Pfalz-LK-Neuwied.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Rheinland-Pfalz-LK-Neuwied.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 322/412 [3565s]
Written file to ipynb/Germany-Rheinland-Pfalz-LK-Rhein-Hunsrück-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Rheinland-Pfalz-LK-Rhein-Hunsrück-Kreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 323/412 [3572s]
Written file to ipynb/Germany-Rheinland-Pfalz-LK-Rhein-Lahn-Kreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Rheinland-Pfalz-LK-Rhein-Lahn-Kreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) no

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 347/412 [3751s]
Written file to ipynb/Germany-Saarland-LK-Stadtverband-Saarbrücken.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Saarland-LK-Stadtverband-Saarbrücken.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 348/412 [3759s]
Written file to ipynb/Germany-Sachsen-LK-Bautzen.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Sachsen-LK-Bautzen.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 349/412 [3766s]
Written file to ipynb/Germany-Sachsen-LK-Erzgebirgskreis.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Sachsen-LK-Erzgebirgskreis.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()

Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Sachsen-Anhalt-SK-Halle.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 374/412 [4856s]
Written file to ipynb/Germany-Sachsen-Anhalt-SK-Magdeburg.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Sachsen-Anhalt-SK-Magdeburg.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 375/412 [4863s]
Written file to ipynb/Germany-Schleswig-Holstein-LK-Dithmarschen.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Schleswig-Holstein-LK-Dithmarschen.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 376/412 [4871s]
Written file to ipynb/Germany-Schleswig-Holstein-LK-Herzogtum-Lauenburg.ipynb
Command = jupyter-nbconvert "wwwroot/ipy

Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 400/412 [5048s]
Written file to ipynb/Germany-Thüringen-LK-Saalfeld-Rudolstadt.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Thüringen-LK-Saalfeld-Rudolstadt.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 401/412 [5055s]
Written file to ipynb/Germany-Thüringen-LK-Schmalkalden-Meiningen.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Thüringen-LK-Schmalkalden-Meiningen.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versions in{time.time()-start_time} seconds}
Processing 402/412 [5063s]
Written file to ipynb/Germany-Thüringen-LK-Sonneberg.ipynb
Command = jupyter-nbconvert "wwwroot/ipynb/Germany-Thüringen-LK-Sonneberg.ipynb"  --to html --execute --output-dir wwwroot/html
Create {len(subregions) notebooks and html versi

In [28]:
c, d = germany_get_region(landkreis='LK Göttingen (alt)')

In [31]:
c2, d2 = germany_get_region(landkreis='LK Göttingen')